In [2]:
import numpy as np
import pandas as pd

from qiskit_aer import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp


# ✅ Step 1: Generate Simulated Database Metrics
def generate_system_metrics(num_samples=100):
    """Simulates database health metrics over time."""
    np.random.seed(42)
    data = []
    
    for _ in range(num_samples):
        cpu_load = np.random.uniform(10, 95)  # CPU % usage
        disk_io = np.random.uniform(50, 500)  # Disk I/O in MB/s
        query_latency = np.random.uniform(5, 500)  # Latency in ms
        active_connections = np.random.randint(10, 1000)  # Open connections

        # Simulate a crash event with some randomness
        crash = 1 if (cpu_load > 85 and query_latency > 400) else np.random.choice([0, 1], p=[0.9, 0.1])

        data.append([cpu_load, disk_io, query_latency, active_connections, crash])

    return pd.DataFrame(data, columns=["cpu_load", "disk_io", "query_latency", "active_connections", "crash"])

# ✅ Step 2: Quantum Crash Probability Model
def quantum_crash_prediction(cpu, disk, latency, connections):
    """Uses a quantum variational model to predict database crash probability."""
    num_qubits = 4  # Encoding CPU, Disk I/O, Latency, Connections
    ansatz = RealAmplitudes(num_qubits, reps=2)

    # Define Hamiltonian (`H`) for crash risk
    H = SparsePauliOp(
        ["ZIII", "IZII", "IIZI", "IIIZ"],  
        coeffs=np.array([
            cpu / 100,       # Higher CPU increases risk
            -disk / 500,     # More disk I/O reduces risk
            latency / 500,   # Higher latency increases risk
            connections / 1000  # More active connections increase risk
        ])
    )

    optimizer = COBYLA(maxiter=100)
    backend = Aer.get_backend('aer_simulator')

    from qiskit.primitives import Estimator
    vqe = VQE(Estimator(), ansatz, optimizer=optimizer)

    result = vqe.compute_minimum_eigenvalue(H)
    
    # Map the quantum energy output to a crash probability (0-1)
    crash_probability = np.clip(result.optimal_value, 0.0, 1.0)
    
    return crash_probability

# ✅ Step 3: Evaluate Crash Probability Over Time
def evaluate_crash_risk(df):
    """Runs quantum crash probability prediction on simulated data."""
    df["quantum_crash_score"] = df.apply(
        lambda row: quantum_crash_prediction(row["cpu_load"], row["disk_io"], row["query_latency"], row["active_connections"]), axis=1
    )

    return df

# ✅ Step 4: Run the Simulation
if __name__ == "__main__":
    print("🔹 Generating Database Health Metrics...")
    system_metrics = generate_system_metrics()

    print("🔹 Running Quantum Prediction for Crash Risk...")
    system_metrics_with_risk = evaluate_crash_risk(system_metrics)

    print("\n✅ Sample of Predicted Crash Probabilities:")
    print(system_metrics_with_risk.head(10))

    # Analyze Quantum Crash Predictions
    avg_crash_risk = system_metrics_with_risk["quantum_crash_score"].mean()
    print(f"\n🚀 Average Quantum-Predicted Crash Probability: {avg_crash_risk:.4f}")

    high_risk_periods = system_metrics_with_risk[system_metrics_with_risk["quantum_crash_score"] > 0.7]
    print(f"\n⚠️ {len(high_risk_periods)} High-Risk Database Periods Identified!")


🔹 Generating Database Health Metrics...
🔹 Running Quantum Prediction for Crash Risk...


/tmp/ipykernel_2899713/2027013822.py:51: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)
/tmp/ipykernel_2899713/2027013822.py:51: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)
/tmp/ipykernel_2899713/2027013822.py:51: DeprecationWarning: The class ``qiskit.primitives.estimator


✅ Sample of Predicted Crash Probabilities:
    cpu_load     disk_io  query_latency  active_connections  crash  \
0  41.835910  477.821438     367.337001                 710      0   
1  47.895784   94.988712     232.328202                 382      0   
2  70.186169   59.263022     485.105377                 501      1   
3  10.066195  496.495202     310.653347                 323      0   
4  46.715327  181.053113     307.867183                 691      0   
5  92.769219  154.747103      49.850185                 967      0   
6  53.709927  316.586556      27.992954                 828      0   
7  48.292436   55.969233     471.389869                  23      0   
8  35.892170   93.952451     343.695348                 376      0   
9  61.849716  424.937710      90.815503                  90      0   

   quantum_crash_score  
0                  0.0  
1                  0.0  
2                  0.0  
3                  0.0  
4                  0.0  
5                  0.0  
6         